In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/nlp'

Mounted at /content/drive
/content/drive/My Drive/nlp


In [ ]:
import string
import numpy as np

## tworzenie potrzebnych słowników

In [ ]:
def create_PT():
  corpus = []
  #global LEN_UNI
  with open('/content/drive/My Drive/nlp/pan-tadeusz.txt', 'r', encoding = 'utf8') as file:
   for i, line in enumerate(file): 
    if i<18:
      continue
    elif i >10823:
      continue
    else:
      if line=='\n':
        continue
      if line =='':
        continue
      line=line.replace('—', '')
      line=line.replace('…', '')
      line=line.replace('»', '')
      line=line.replace('é', 'e')
      line=line.replace('«', '')
      line=line.replace('à', '')
      line=line.replace('æ', '')

      words = line.split()
      corpus.append(line)
  return corpus

corpus_PT = create_PT()
corpus_PT[1:20]

['Ile cię trzeba cenić, ten tylko się dowie,\n',
 'Kto cię stracił. Dziś piękność twą w całej ozdobie\n',
 'Widzę i opisuję, bo tęsknię po tobie.\n',
 '    Panno święta, co Jasnej bronisz Częstochowy\n',
 'I w Ostrej świecisz Bramie! Ty, co gród zamkowy\n',
 'Nowogródzki ochraniasz z jego wiernym ludem!\n',
 'Jak mnie dziecko do zdrowia powróciłaś cudem\n',
 '(Gdy od płaczącej matki, pod Twoją opiekę\n',
 'Ofiarowany, martwą podniosłem powiekę;\n',
 'I zaraz mogłem pieszo, do Twych świątyń progu\n',
 'Iść za wrócone życie podziękować Bogu),\n',
 'Tak nas powrócisz cudem na Ojczyzny łono.\n',
 'Tymczasem przenoś moją duszę utęsknioną\n',
 'Do tych pagórków leśnych, do tych łąk zielonych,\n',
 'Szeroko nad błękitnym Niemnem rozciągnionych;\n',
 'Do tych pól malowanych zbożem rozmaitem,\n',
 'Wyzłacanych pszenicą, posrebrzanych żytem;\n',
 'Gdzie bursztynowy świerzop, gryka jak śnieg biała,\n',
 'Gdzie panieńskim rumieńcem dzięcielina pała,\n']

In [ ]:
def create_corpus():
  corpus = {}
  with open('/content/drive/My Drive/nlp/polish_corpora.txt', 'r', encoding = 'utf8') as file:
   for i, line in enumerate(file):
      words = line.split()
      for word in words:
        if word in corpus:
          num = corpus[word]
          corpus[word] = num + 1
        else:
          corpus[word] = 1
  return corpus

corpus = create_corpus()

In [ ]:
def create_tags():
  tags = {}
  with open('/content/drive/My Drive/nlp/supertags.txt', 'r', encoding = 'utf8') as file:
    for line in file:
      words = line.split()
      tags[words[0]] = words[1]
  return tags
  
tags = create_tags()

In [ ]:
def create_tags_unigrams(corpus, tags_corp): 
  tags_unigrams = {}
  for word in corpus:
    if word in tags_corp:
      tag = tags_corp[word]
      if tag in tags_unigrams:
        tags_unigrams[tag].append((word, corpus[word]))
      else:
        tags_unigrams[tag] = [(word, corpus[word])]
    else:
      new_tag = ('^' + word)[-3:]
      tags_unigrams[new_tag] = [(word, corpus[word])]
  return tags_unigrams
  
tags_unigrams = create_tags_unigrams(corpus, tags)

In [ ]:
corpus['koń']

1614

### kilka funkcji pomocniczych oraz prepocessing

In [ ]:
def divide_syllable(word):
  vowel = ['a','ą','e','ę', 'i','o','u','y','ó']
  vowel_count=0
  prev_i = -2
  for i in range(len(word)):
    if word[i] in vowel:
      if prev_i +1 == i:
        prev_i = -2
        continue
      if word[i] == 'i':
        prev_i = i
      vowel_count+=1
  return vowel_count

In [ ]:
def normalization_dict(values, val_id):
  frequency = []
  for x in values:
    frequency.append((x[val_id]))
  sum_all = sum(frequency)
  return [i/sum_all for i in frequency]

In [ ]:
def check_rhyme(word1, word2):
  vowel = ['a','ą','e','ę', 'i','o','u','y','ó']
  vowel_count = 0
  i= len(word1)-1
  last_vowel = ''
  last_vowel_pos = -1
  while 1:
    if i == -1: break 
    if word1[i] in vowel:
      if word1[i] == 'i' and last_vowel_pos == i+1:
        i-=1
        continue
      last_vowel = word1[i]
      last_vowel_pos = i
      vowel_count+=1
      if vowel_count == 2:
        break
    i-=1
  j = len(word1) - i
  if word1[i:] == word2[-j:]:
    return True
  else:
    return False
check_rhyme('jula', 'gsfgsgbula')

True

In [ ]:
def verse_prepocessing(verse1, verse2):
  if verse1==[] or verse2==[]:
    return (False, False)
  verse1 = verse1.translate(str.maketrans('', '', string.punctuation))
  verse1 = verse1.lower()
  words_verse1 = verse1.split()

  verse2 = verse2.translate(str.maketrans('', '', string.punctuation))
  verse2 = verse2.lower()
  words_verse2 = verse2.split()
  if len(words_verse1) > 10 or len(words_verse2) > 10:
    return (False, False)

  if not check_rhyme(words_verse1[-1:][0], words_verse2[-1:][0]):
    return (False, False)
  verse1_tag, verse2_tag = [], []
  for word in words_verse1:
    if word in tags:
      verse1_tag.append(tags[word])
    else:
      return (False, False)
  for word in words_verse2:
    if word in tags:
      verse2_tag.append(tags[word])
    else:
      return (False, False)
  return ([words_verse1, words_verse2], [verse1_tag, verse2_tag])

In [ ]:
verse_prepocessing(corpus_PT[10],corpus_PT[11])

([['i', 'zaraz', 'mogłem', 'pieszo', 'do', 'twych', 'świątyń', 'progu'],
  ['iść', 'za', 'wrócone', 'życie', 'podziękować', 'bogu']],
 [['S65', 'T3161', 'T1337', 'T21', 'T2679', 'T2768', 'T3194', 'T3662'],
  ['T703', 'S234', 'T2257', 'T3614', 'T857', 'T1694']])

In [ ]:
for i in range(20):
  a,b = verse_prepocessing(corpus_PT[i],corpus_PT[i+1])
  if a!= False: 
    print(a,b)

[['litwo', 'ojczyzno', 'moja', 'ty', 'jesteś', 'jak', 'zdrowie'], ['ile', 'cię', 'trzeba', 'cenić', 'ten', 'tylko', 'się', 'dowie']] [['T1514', 'T1514', 'T2310', 'T2174', 'T1316', 'T251', 'T80'], ['T3725', 'T66', 'T2115', 'T1085', 'T245', 'S193', 'S167', 'T234']]
[['kto', 'cię', 'stracił', 'dziś', 'piękność', 'twą', 'w', 'całej', 'ozdobie'], ['widzę', 'i', 'opisuję', 'bo', 'tęsknię', 'po', 'tobie']] [['T2750', 'T66', 'T278', 'T21', 'T3046', 'T1089', 'S229', 'T1883', 'T2957'], ['T1044', 'S65', 'T2693', 'S23', 'T3422', 'S229', 'T2524']]
[['panno', 'święta', 'co', 'jasnej', 'bronisz', 'częstochowy'], ['i', 'w', 'ostrej', 'świecisz', 'bramie', 'ty', 'co', 'gród', 'zamkowy']] [['T1514', 'T108', 'T3345', 'T1883', 'T91', 'T3777'], ['S65', 'S229', 'T1883', 'T91', 'T2957', 'T2174', 'T3345', 'T3168', 'T3038']]
[['nowogródzki', 'ochraniasz', 'z', 'jego', 'wiernym', 'ludem'], ['jak', 'mnie', 'dziecko', 'do', 'zdrowia', 'powróciłaś', 'cudem']] [['T3038', 'T91', 'S232', 'T2150', 'T3167', 'T46'], ['T

## zad 6 unigramy

In [ ]:
all, score = 0, 0
def generate_PT_similar(n):
  global all, score
  verse1 = corpus_PT[n]
  verse2 = corpus_PT[n+1]
  
  verses, verses_tags = verse_prepocessing(verse1, verse2)
  if verses == False:
    return 'null'
  all+=1
  words_verse1 = verses[0]
  words_verse2 = verses[1]
  verse1_similar, verse2_similar = [], []
  
  for word in words_verse1:
    syllable_count = divide_syllable(word)
    next_word = False
    repeat = 200
    word_tag = tags[word]
    value_elements = len(tags_unigrams[word_tag])
    weights = normalization_dict(tags_unigrams[word_tag], 1)
    while repeat>0:
      chosen_word = np.random.choice(value_elements, p=weights)
      next = tags_unigrams[word_tag][chosen_word-1][0]
      if syllable_count == divide_syllable(next):
        verse1_similar.append(next)
        next_word = True
        break
      else:
        repeat -=1
    if next_word == False:
      return 'null'

  for word in words_verse2:
    syllable_count = divide_syllable(word)
    next_word = False
    repeat = 200
    word_tag = tags[word]
    value_elements = len(tags_unigrams[word_tag])
    weights = normalization_dict(tags_unigrams[word_tag], 1)
    while repeat>0:
      chosen_word = np.random.choice(value_elements, p=weights)
      next = tags_unigrams[word_tag][chosen_word-1][0]
      if syllable_count == divide_syllable(next):
        verse2_similar.append(next)
        next_word = True
        break
      else:
        repeat -=1
    if next_word == False:
      return 'null'

  same_rhyme = False
  repeat = 200
  rhyme_correct = False
  syllable_count = divide_syllable(verse1_similar[-1:][0])
  word = verse1_similar[-1:][0]
  word_tag = tags[word]
  value_elements = len(tags_unigrams[word_tag])
  weights = normalization_dict(tags_unigrams[word_tag], 1)

  if check_rhyme(verse1_similar[-1:][0],verse2_similar[-1:][0]):
    print(verse1_similar)
    print(verse2_similar)
    score+=1
    return (verse1_similar, verse2_similar)
  else:
    del verse2_similar[-1:]
    while repeat > 0:
      chosen_word = np.random.choice(value_elements, p=weights)
      next_word = tags_unigrams[word_tag][chosen_word-1][0]
      if syllable_count == divide_syllable(next_word) and check_rhyme(verse1_similar[-1:][0], next_word):
        if next_word == verse1_similar[-1:][0]:
          same_rhyme = True
        else: 
          score+=1
          rhyme_correct = True
          verse2_similar.append(next_word)
          print(verse1_similar)
          print(verse2_similar)
      if rhyme_correct:
        break
      repeat-=1
    if same_rhyme and not rhyme_correct:
      score+=1
      verse2_similar.append(verse1_similar[-1:][0])
      print(verse1_similar)
      print(verse2_similar)
      return (verse1_similar, verse2_similar)
  return 'null'


 (tag1,word1) : [(tag2, word2, count),(tag2, word2, count) ...]

In [ ]:
for i in np.arange(0,300,1):
  #print(i)
  generate_PT_similar(i)
print(score, all)

same same
['tym', 'wyższe', 'że', 'wpłacone', 'od', 'czeskiej', 'zieleni']
['subwencji', 'co', 'go', 'leją', 'od', 'zgonów', 'zieleni']
['też', 'swoje', 'cieka', 'diesle', 'też', 'same', 'wchodzenia']
['z', 'naszymi', 'się', 'odcinać', 'stykał', 'bez', 'znoszenia']
['ki', 'plackarnio', 'w', 'dziedzinie', 'wewnętrznej', 'z', 'drzewkami']
['zdobywanymi', 'w', 'plwanie', 'miecz', 'kwarkowo', 'mediami']
same same
['po', 'roku', 'długim', 'brzeskim', 'brzeskim', 'po', 'młyn', 'zysku']
['film', 'statyczny', 'lecz', 'płocki', 'zasadzisz', 'że', 'w', 'zysku']
same same
['w', 'jednym', 'podstawa', 'tylko', 'słyszeć', 'zwykła', 'z', 'fanga']
['w', 'którym', 'krnąbrnie', 'nie', 'cieszy', 'od', 'ludzi', 'fanga']
same same
['bukwi', 'nie', 'było', 'widać', 'przyjęta', 'po', 'morze']
['stwierdzała', 'kogóż', 'wejściem', 'skarbowo', 'po', 'morze']
same same
['radni', 'nie', 'wyszli', 'głosić', 'ależ', 'nie', 'myśl', 'mącznie']
['aby', 'w', 'skoku', 'prezesa', 'wmawiano', 'mącznie']
same same
['do', '

## zadanie 6 bigramy

In [ ]:
def create_bigrams():
  bigrams_dict = {}
  with open('/content/drive/My Drive/nlp/poleval_2grams.txt', 'r', encoding = 'utf8') as file:
    for i, line in enumerate(file):
      words = line.split()
      how_many, word1, word2 = int(words[0]), words[1], words[2]  #cast to int
      if word1 in tags:
        tag1 = tags[word1]
      else: continue
      if word2 in tags: 
        tag2 = tags[word2]
      else: continue

      if (tag1, word1) in bigrams_dict:
        bigrams_dict[(tag1, word1)].append((tag2, word2, how_many))
      else:
        bigrams_dict[(tag1, word1)] = [(tag2, word2, how_many)]
  return bigrams_dict
bigrams_tags = create_bigrams()

In [ ]:
len(bigrams_tags)

649231

In [ ]:
k=0
for i in bigrams_tags:
  print(i,len(bigrams_tags[i]), bigrams_tags[i])
  if k==10: break
  k+=1

('T722', 'operatora') 920 [('T3272', 'zarówno', 1), ('T2586', 'aparatu', 1), ('T326', 'platformy', 5), ('T2800', 'rewitalizacji', 2), ('T779', 'polecacie', 1), ('T2586', 'poboru', 2), ('T3215', 'zadań', 1), ('T21', 'ponownie', 2), ('T861', 'parzystości', 1), ('T3452', 'piotrka', 1), ('T3157', 'wartością', 1), ('S229', 'nad', 3), ('T582', 'nam', 1), ('T441', 'reprezentowanego', 2), ('T3662', 'pliku', 1), ('T3601', 'filmowego', 47), ('T94', 'chodzi', 1), ('T3362', 'delta', 1), ('T3662', 'brzegu', 1), ('T3601', 'komercyjnego', 3), ('T1373', 'definiuje', 2), ('S229', 'ponad', 1), ('T3194', 'kamer', 1), ('T632', 'okaże', 1), ('T3601', 'kablowego', 8), ('T1516', 'składająca', 1), ('T3601', 'lotniczego', 4), ('T1806', 'zastrzegliście', 1), ('T3093', 'lottomatów', 4), ('T3015', 'wysypiska', 1), ('T3015', 'uregulowania', 1), ('T3174', 'realizującego', 2), ('T2800', 'telefonii', 40), ('T2800', 'radiotelekomunikacji', 1), ('T2468', 'pozwala', 2), ('T3601', 'intermodalnego', 1), ('T3601', 'telekom

In [ ]:
def find_first_with_unigrams(word, word_tag, num_rep=200):
  syllable_count = divide_syllable(word)
  value_elements = len(tags_unigrams[word_tag])
  weights = normalization_dict(tags_unigrams[word_tag], 1)
  while num_rep>0:
    chosen_word = np.random.choice(value_elements, p=weights)
    next = tags_unigrams[word_tag][chosen_word-1][0]
    if syllable_count == divide_syllable(next):
      return next
    else:
        num_rep -=1
  
  for words in  tags_unigrams[word_tag]:
    next = words[0]
    if syllable_count == divide_syllable(next):
      return next
  return False

In [ ]:
def generate_verse_similar(words_verse, verse_similar, allowed, bigrams):
  word = words_verse[0]
  next = find_first_with_unigrams(word, tags[word]) #losujemy pierwsze słowo, unigramy
  if next == False:
    return False, False
  verse_similar.append(next)
  tag_prev = tags[word]
  word_prev = word
  backtrack, i = 0, 1
  while i < len(words_verse): #pozostale slowa
    if i==0:
      verse_similar = []
      word = words_verse[0]
      next = find_first_with_unigrams(word, tags[word])
      verse_similar.append(next)
      tag_prev = tags[word]
      word_prev = word
      i = 1

    if backtrack == 50: #budujemy od poczatku
      verse_similar = []
      backtrack, i = 0, 1
      word = words_verse[0]
      next = find_first_with_unigrams(word, tags[word])
      verse_similar.append(next)
      tag_prev = tags[word]
      word_prev = word
      allowed-=1
      if allowed == 0: return False, False
      continue

    word_curr = words_verse[i]
    tag_curr = tags[word_curr]
    syllable_count = divide_syllable(word_curr)
    correct_tags_bigrams = [] 
    if (tag_prev, word_prev) not in bigrams:
      i-=1
      del verse_similar[-1:]
      backtrack+=1
      if i==-1:
        i=0
        continue
      tag_prev = tags[words_verse[i]]
      word_prev = words_verse[i]
      continue      

    for k in bigrams[(tag_prev, word_prev)]: #budujemy tablice dobrych slow
      if k[0]==tag_curr and syllable_count == divide_syllable(k[1]):
        correct_tags_bigrams.append(k)
    if correct_tags_bigrams == []:
      i-=1
      del verse_similar[-1:]
      backtrack+=1
      if i==-1:
        i=0
        continue
      tag_prev = tags[words_verse[i]]
      word_prev = words_verse[i]
      continue

    value_elements = len(correct_tags_bigrams)
    weights = normalization_dict(correct_tags_bigrams, 2)
    chosen_word = np.random.choice(value_elements, p=weights) #losujemy tylko z poprawnych slow
    tag_next, next, _ = correct_tags_bigrams[chosen_word-1] 
    verse_similar.append(next)
    tag_prev = tag_curr
    word_prev = next
    i+=1
  return verse_similar, allowed

In [ ]:
def generate_PT_similar_bigrams(n, bigrams):
  new_allowed = 5
  verse1, verse2 = corpus_PT[n], corpus_PT[n+1]
  verses, verses_tags = verse_prepocessing(verse1, verse2)
  if verses == False:
    return 'null'
  words_verse1, words_verse2 = verses[0], verses[1]
  verse1_similar, verse2_similar = [], []

  verse1_similar, new_allowed = generate_verse_similar(words_verse1, verse1_similar, new_allowed, bigrams)
  if verse1_similar == False:
    return 'null'

  verse2_similar, new_allowed = generate_verse_similar(words_verse2, verse2_similar, new_allowed, bigrams)
  if verse2_similar == False:
    return 'null'
  repeat = 200

  syllable_count = divide_syllable(words_verse2[-1:][0])
  correct_tags_bigrams = []
  for k in bigrams[(tags[verse2_similar[-2:][0]],  verse2_similar[-2:][0] )]: #budujemy tablice dobrych slow
    if k[0]==tags[verse2_similar[-1:][0]] and syllable_count == divide_syllable(k[1]):
      correct_tags_bigrams.append(k)

  syllable_count = divide_syllable(verse2_similar[-1:][0])
  word_to_rhyme = verse1_similar[-1:][0]
  word_tag = tags[verse2_similar[-1:][0]]
  
  if check_rhyme(verse1_similar[-1:][0],verse2_similar[-1:][0]):
    print('phrase', n)
    print(words_verse1)
    print(words_verse2)
    print(verse1_similar)
    print(verse2_similar)
    return (words_verse1, words_verse2)
  else:
    del verse2_similar[-1:]
    value_elements = len(correct_tags_bigrams)
    weights = normalization_dict(correct_tags_bigrams, 2)
    while repeat > 0:
      chosen_word = np.random.choice(value_elements, p=weights) #losujemy tylko z poprawnych slow
      tag_next, next, _ = correct_tags_bigrams[chosen_word-1] 
      if check_rhyme(verse1_similar[-1:][0], next):
        verse2_similar.append(next)
        print('phrase', n)
        print(words_verse1)
        print(words_verse2)
        print(verse1_similar)
        print(verse2_similar)
        return (words_verse1, words_verse2)
      repeat-=1
  if repeat==0:
    return 'null'

In [ ]:
for i in np.arange(0,100,1):
  generate_PT_similar_bigrams(i, bigrams_tags)

phrase 24
['stał', 'dwór', 'szlachecki', 'z', 'drzewa', 'lecz', 'podmurowany']
['świeciły', 'się', 'z', 'daleka', 'pobielane', 'ściany']
['słał', 'gród', 'wawelski', 'z', 'miejsca', 'lecz', 'atakowany']
['wspierały', 'się', 'z', 'ostatnia', 'stosowane', 'zmiany']
phrase 58
['takim', 'był', 'gdy', 'przysięgał', 'na', 'stopniach', 'ołtarzów']
['że', 'tym', 'mieczem', 'wypędzi', 'z', 'polski', 'trzech', 'mocarzów']
['wszystkim', 'był', 'gdy', 'porzucał', 'po', 'wierszach', 'fragmentów']
['że', 'swym', 'wzorem', 'przyciągnie', 'z', 'drogi', 'trzech', 'pacjentów']


In [ ]:
for i in np.arange(0,500,1):
  generate_PT_similar_bigrams(i, bigrams_tags)

phrase 141
['tymczasem', 'na', 'folwarku', 'nie', 'uszło', 'baczności']
['że', 'przed', 'ganek', 'zajechał', 'któryś', 'z', 'nowych', 'gości']
['szanownie', 'pod', 'podatku', 'nie', 'wzmogło', 'trudności']
['że', 'pod', 'pomnik', 'powrócił', 'jakiś', 'z', 'ważnych', 'gości']
phrase 179
['a', 'starzy', 'i', 'kobiety', 'żniwo', 'oglądają']
['pod', 'lasem', 'i', 'tam', 'pewnie', 'na', 'młodzież', 'czekają']
['a', 'starzy', 'i', 'sukienki', 'lato', 'nabierają']
['po', 'trzonem', 'i', 'płyt', 'bardzo', 'pod', 'jesień', 'spadają']
phrase 227
['krótkie', 'były', 'sędziego', 'z', 'synowcem', 'witania']
['dał', 'mu', 'poważnie', 'rękę', 'do', 'pocałowania']
['sprzeczne', 'były', 'ministra', 'z', 'technikiem', 'leśnictwa']
['zdał', 'mu', 'serdecznie', 'hannę', 'do', 'białoskórnictwa']
phrase 454
['jeśli', 'kto', 'i', 'czuł', 'wtenczas', 'że', 'polskie', 'ubranie']
['piękniejsze', 'jest', 'niż', 'obcej', 'mody', 'małpowanie']
['póty', 'kto', 'i', 'czuł', 'bardzo', 'że', 'względne', 'pierwszeństwo

In [ ]:
for i in np.arange(5000,5500,1):
  generate_PT_similar_bigrams(i, bigrams_tags)

phrase 5041
['wewnątrz', 'samego', 'domu', 'w', 'stajni', 'i', 'wozowni']
['pełno', 'znajdziesz', 'rynsztunków', 'jak', 'w', 'starej', 'zbrojowni']
['mącznie', 'tamtego', 'ruchu', 'w', 'myjni', 'i', 'lokacji']
['wszędzie', 'znajdziesz', 'przypadków', 'jak', 'w', 'smutnej', 'tonacji']
phrase 5078
['lecz', 'gdy', 'król', 'w', 'targowicy', 'przyjął', 'uczestnictwo']
['maciej', 'opuścił', 'znowu', 'królewskie', 'stronnictwo']
['lecz', 'gdy', 'król', 'w', 'kamienicy', 'zaczął', 'wydawanie']
['lekarz', 'zgromadził', 'bardzo', 'kolejne', 'przyznanie']
phrase 5255
['maciej', 'głowy', 'nie', 'ruszył', 'ani', 'podniósł', 'wzroku']
['tylko', 'ręką', 'kilkakroć', 'uderzył', 'po', 'boku']
['rotmistrz', 'żaby', 'nie', 'spojrzał', 'ani', 'zamknął', 'dachu']
['tylko', 'opcją', 'faktycznie', 'obudził', 'po', 'krachu']
phrase 5267
['cicho', 'skądże', 'ta', 'cała', 'nowina', 'pochodzi']
['jak', 'daleko', 'francuzi', 'kto', 'nimi', 'dowodzi']
['świńsko', 'kiedy', 'ta', 'carska', 'komisja', 'powraca']
['ja

In [ ]:
for i in np.arange(7000,8000,1):
  generate_PT_similar_bigrams(i, bigrams_tags)

phrase 7032
['co', 'potem', 'dokazywał', 'jakie', 'klęski', 'szerzył']
['daremnie', 'śpiewać', 'nikt', 'by', 'muzie', 'nie', 'uwierzył']
['co', 'czasem', 'decydował', 'jakie', 'floty', 'służył']
['skromniutko', 'szukać', 'kto', 'by', 'izbie', 'nie', 'przysłużył']
phrase 7203
['mam', 'dawniejszą', 'urazę', 'on', 'do', 'zamku', 'mego']
['mów', 'pan', 'przerwał', 'protazy', 'do', 'zamku', 'naszego']
['mam', 'wcześniejszą', 'piosenkę', 'on', 'znad', 'dachu', 'swego']
['trzeb', 'lech', 'przeniósł', 'konarski', 'do', 'druku', 'swojego']
phrase 7332
['wyszedł', 'wreszcie', 'ujrzawszy', 'że', 'było', 'po', 'bitwie']
['taki', 'miał', 'koniec', 'zajazd', 'ostatni', 'na', 'litwie']
['zmalał', 'dużo', 'zważywszy', 'że', 'wstało', 'na', 'ramce']
['któryż', 'miał', 'eksport', 'import', 'przestępczy', 'na', 'ramce']
phrase 7345
['jak', 'wielki', 'żagiel', 'biorąc', 'wszystkie', 'wiatry', 'w', 'siebie']
['od', 'południa', 'na', 'zachód', 'leciała', 'po', 'niebie']
['jak', 'nyski', 'szpital', 'tworząc'

## zadanie 7 nowa struktura z ppmi

In [ ]:
k=0
for word in corpus:
  print(word, corpus[word])
  k+=1
  if k==10: break

System 17556
Euroraty 1
Chcesz 2145
kupować 4089
więcej 132096
niż 228834
gdzie 260628
indziej 3875
? 873647
Parlament 3331


In [ ]:
k=0
for word in bigrams_tags:
  print(word, bigrams_tags[word])
  k+=1
  if k==10: break

('T722', 'operatora') [('T3272', 'zarówno', 1), ('T2586', 'aparatu', 1), ('T326', 'platformy', 5), ('T2800', 'rewitalizacji', 2), ('T779', 'polecacie', 1), ('T2586', 'poboru', 2), ('T3215', 'zadań', 1), ('T21', 'ponownie', 2), ('T861', 'parzystości', 1), ('T3452', 'piotrka', 1), ('T3157', 'wartością', 1), ('S229', 'nad', 3), ('T582', 'nam', 1), ('T441', 'reprezentowanego', 2), ('T3662', 'pliku', 1), ('T3601', 'filmowego', 47), ('T94', 'chodzi', 1), ('T3362', 'delta', 1), ('T3662', 'brzegu', 1), ('T3601', 'komercyjnego', 3), ('T1373', 'definiuje', 2), ('S229', 'ponad', 1), ('T3194', 'kamer', 1), ('T632', 'okaże', 1), ('T3601', 'kablowego', 8), ('T1516', 'składająca', 1), ('T3601', 'lotniczego', 4), ('T1806', 'zastrzegliście', 1), ('T3093', 'lottomatów', 4), ('T3015', 'wysypiska', 1), ('T3015', 'uregulowania', 1), ('T3174', 'realizującego', 2), ('T2800', 'telefonii', 40), ('T2800', 'radiotelekomunikacji', 1), ('T2468', 'pozwala', 2), ('T3601', 'intermodalnego', 1), ('T3601', 'telekomunik

In [ ]:
def uni_bi_tags_len():
  LEN_UNI = 0
  LEN_BI = 0
  i=0
  for word in corpus:
    LEN_UNI += corpus[word]
  for tag_and_word in bigrams_tags:
    for list_words in bigrams_tags[tag_and_word]:
      LEN_BI += list_words[2]
  print(LEN_UNI, LEN_BI)
  return LEN_UNI, LEN_UNI
LEN_UNI, LEN_BI = uni_bi_tags_len()

451846640 286699087


In [ ]:
def create_bigrams_ppmi():
  bigrams_dict = {}
  i=0
  for tag_and_word in bigrams_tags:
    i+=1
    if tag_and_word[1] not in corpus:
      corpus[tag_and_word[1]] = 1
    j = 0
    for list_words in bigrams_tags[tag_and_word]:
      if list_words[1] not in corpus:
        corpus[list_words[1]] = 1
      ppmi = max(0, np.log(list_words[2]/ (corpus[tag_and_word[1]] * corpus[list_words[1]]) *LEN_UNI**2 /LEN_BI ))

      if (tag_and_word[0], tag_and_word[1]) in bigrams_dict:
        bigrams_dict[(tag_and_word[0], tag_and_word[1])].append((list_words[0], list_words[1], ppmi+1e-100))
      else:
        bigrams_dict[(tag_and_word[0], tag_and_word[1])] = [(list_words[0], list_words[1], ppmi+1e-100)]
  return bigrams_dict
bigrams_ppmi = create_bigrams_ppmi()

### testy kolokacyjności

In [ ]:
def check_ppmi(phrases_array):
  ppmi_values = []
  for phrase in phrases_array:
    ppmi_phrase = 0
    for i in range(len(phrase)-1):
      if i ==0: continue
      bigrams_occ = 1e-100
      if (tags[phrase[i]], phrase[i]) not in bigrams_tags:
        continue 
      for  word1 in bigrams_tags[(tags[phrase[i]], phrase[i] ) ]:
        if word1[1] == phrase[i+1]:
          bigrams_occ = word1[2]
          break
      if phrase[i] not in corpus:
        corpus[phrase[i]] = bigrams_occ          
      if phrase[i+1] not in corpus:
        corpus[phrase[i+1]] = bigrams_occ
      ppmi_phrase += max(0, np.log(bigrams_occ/ (corpus[phrase[i]] * corpus[phrase[i+1]]) *LEN_UNI**2 /LEN_BI ))
    ppmi_values.append(ppmi_phrase)
  return ppmi_values

In [ ]:
created_ppmi = []
for i in np.arange(0,500,1):
  arr = generate_PT_similar_bigrams(i, bigrams_ppmi)
  if arr != 'null':
    created_ppmi.append(arr[0])
    created_ppmi.append(arr[1])

phrase 24
['stał', 'dwór', 'szlachecki', 'z', 'drzewa', 'lecz', 'podmurowany']
['świeciły', 'się', 'z', 'daleka', 'pobielane', 'ściany']
['stał', 'schron', 'cywilny', 'z', 'miejsca', 'lecz', 'zobowiązany']
['zbierały', 'się', 'z', 'daleka', 'pobielane', 'ściany']
phrase 141
['tymczasem', 'na', 'folwarku', 'nie', 'uszło', 'baczności']
['że', 'przed', 'ganek', 'zajechał', 'któryś', 'z', 'nowych', 'gości']
['żelaźnie', 'na', 'majątku', 'nie', 'wzmogło', 'trudności']
['że', 'przed', 'laser', 'zadziałał', 'jakiś', 'z', 'szkolnych', 'gości']
phrase 189
['na', 'spoczynek', 'powraca', 'już', 'krąg', 'promienisty']
['spuszcza', 'się', 'na', 'wierzch', 'boru', 'i', 'już', 'pomrok', 'mglisty']
['po', 'element', 'pochodzi', 'już', 'styk', 'kulturowy']
['trąbi', 'się', 'nad', 'miód', 'świata', 'i', 'już', 'ulic', 'nowy']
phrase 227
['krótkie', 'były', 'sędziego', 'z', 'synowcem', 'witania']
['dał', 'mu', 'poważnie', 'rękę', 'do', 'pocałowania']
['inne', 'były', 'ministra', 'z', 'przywódcą', 'stronn

In [ ]:
for i in np.arange(500,1500,1):
  arr = generate_PT_similar_bigrams(i, bigrams_ppmi)
  if arr != 'null':
    created_ppmi.append(arr[0])
    created_ppmi.append(arr[1])

phrase 535
['jej', 'ubiór', 'zwrócił', 'oczy', 'wszyscy', 'ją', 'witali']
['prócz', 'tadeusza', 'widać', 'że', 'ją', 'wszyscy', 'znali']
['jej', 'fundusz', 'zwrócił', 'oczy', 'wszyscy', 'ją', 'puszczali']
['prócz', 'mańkowskiego', 'słychać', 'że', 'nią', 'wszyscy', 'grali']
phrase 555
['trudno', 'było', 'bo', 'krzeseł', 'dla', 'gości', 'nie', 'stało']
['na', 'czterech', 'ławach', 'cztery', 'ich', 'rzędy', 'siedziało']
['świńsko', 'było', 'bo', 'świadectw', 'od', 'gości', 'nie', 'stało']
['po', 'czterech', 'stoczniach', 'cztery', 'ich', 'punkty', 'składało']
phrase 563
['pośliznęła', 'się', 'nieco', 'i', 'w', 'tym', 'roztargnieniu']
['na', 'pana', 'tadeusza', 'wsparła', 'się', 'ramieniu']
['pobeczała', 'się', 'chętnie', 'i', 'w', 'mym', 'wystąpieniu']
['po', 'wnuka', 'kaczyńskiego', 'wsparła', 'się', 'myśleniu']
phrase 623
['ale', 'razem', 'niemałą', 'chętkę', 'do', 'swawoli']
['z', 'góry', 'już', 'robił', 'projekt', 'że', 'sobie', 'pozwoli']
['ależ', 'potem', 'niewielką', 'serię', 'do'

In [ ]:
created_bigrams = []
for i in np.arange(0,500,1):
  arr = generate_PT_similar_bigrams(i, bigrams_tags)
  if arr != 'null':
    created_bigrams.append(arr[0])
    created_bigrams.append(arr[1])

phrase 24
['stał', 'dwór', 'szlachecki', 'z', 'drzewa', 'lecz', 'podmurowany']
['świeciły', 'się', 'z', 'daleka', 'pobielane', 'ściany']
['stał', 'wóz', 'stropowy', 'z', 'miejsca', 'lecz', 'zryczałtowany']
['walały', 'się', 'z', 'daleka', 'pobielane', 'ściany']
phrase 139
['głośno', 'i', 'sam', 'nie', 'wiedział', 'czy', 'go', 'miało', 'śmieszyć']
['to', 'dziwaczne', 'spotkanie', 'czy', 'wstydzić', 'czy', 'cieszyć']
['krnąbrnie', 'i', 'sam', 'nie', 'pragnął', 'czy', 'go', 'miało', 'sięgać']
['toż', 'nachalne', 'narciarstwo', 'czy', 'chodzić', 'tak', 'sięgać']
phrase 141
['tymczasem', 'na', 'folwarku', 'nie', 'uszło', 'baczności']
['że', 'przed', 'ganek', 'zajechał', 'któryś', 'z', 'nowych', 'gości']
['spadzisto', 'przed', 'pałacu', 'nie', 'wzmogło', 'trudności']
['że', 'po', 'transport', 'przyjechał', 'jakiś', 'z', 'szkolnych', 'gości']
phrase 179
['a', 'starzy', 'i', 'kobiety', 'żniwo', 'oglądają']
['pod', 'lasem', 'i', 'tam', 'pewnie', 'na', 'młodzież', 'czekają']
['a', 'starzy', 'i',

In [ ]:
for i in np.arange(500,1500,1):
  arr = generate_PT_similar_bigrams(i, bigrams_tags)
  if arr != 'null':
    created_bigrams.append(arr[0])
    created_bigrams.append(arr[1])

phrase 535
['jej', 'ubiór', 'zwrócił', 'oczy', 'wszyscy', 'ją', 'witali']
['prócz', 'tadeusza', 'widać', 'że', 'ją', 'wszyscy', 'znali']
['niej', 'western', 'zwrócił', 'oczy', 'wszyscy', 'ją', 'czytali']
['do', 'sosnowskiego', 'trzeba', 'że', 'nią', 'wszyscy', 'grali']
phrase 555
['trudno', 'było', 'bo', 'krzeseł', 'dla', 'gości', 'nie', 'stało']
['na', 'czterech', 'ławach', 'cztery', 'ich', 'rzędy', 'siedziało']
['mącznie', 'było', 'bo', 'mistrzostw', 'dla', 'gości', 'nie', 'stało']
['po', 'czterech', 'barwach', 'cztery', 'ich', 'środki', 'stwarzało']
phrase 623
['ale', 'razem', 'niemałą', 'chętkę', 'do', 'swawoli']
['z', 'góry', 'już', 'robił', 'projekt', 'że', 'sobie', 'pozwoli']
['ależ', 'razem', 'niewielką', 'resztę', 'do', 'swawoli']
['z', 'płyty', 'już', 'gościł', 'zespół', 'że', 'sobie', 'pozwoli']
phrase 657
['cóż', 'gdy', 'potem', 'zaczęła', 'mówić', 'o', 'malarstwie']
['o', 'muzyce', 'o', 'tańcach', 'nawet', 'o', 'rzeźbiarstwie']
['cóż', 'gdy', 'razem', 'zaczęła', 'mówić', '

In [ ]:
created_unigrams = []
for i in np.arange(0,500,1):
  arr = generate_PT_similar(i)
  if arr != 'null':
    created_unigrams.append(arr[0])
    created_unigrams.append(arr[1])

['tu', 'fioletowy', 'tlenek', 'furtka', 'jak', 'mim', 'szkolna']
['tak', 'węglowym', 'parnikiem', 'emigracja', 'dolna']
same same
['tym', 'nowsze', 'że', 'zbliżone', 'bez', 'starej', 'przyjaźni']
['pozycji', 'co', 'go', 'wodzą', 'bez', 'węgli', 'przyjaźni']
['tworzących', 'pańsko', 'plany', 'bezbarwne', 'azotu']
['dynastyczne', 'indyjsko', 'pochodne', 'do', 'przedmiotu']
same same
['też', 'same', 'cieka', 'plany', 'też', 'same', 'ryzyka']
['z', 'którymi', 'się', 'pozdrawiać', 'lubił', 'od', 'ryzyka']
['tu', 'posłanko', 'w', 'odsłonie', 'poselskiej', 'z', 'uszami']
['rozwodnionymi', 'w', 'dnienie', 'tekst', 'islamsko', 'oczkami']
['po', 'biegu', 'świetlnym', 'znacznym', 'brzeskim', 'po', 'skład', 'roku']
['zbór', 'genialny', 'lecz', 'znaczny', 'przekupisz', 'że', 'w', 'roku']
['hitowy', 'krnąbrnie', 'w', 'futrze', 'słał', 'czyniąc', 'smakując']
['złotymi', 'wyrobami', 'wzorów', 'promując']
same same
['w', 'samym', 'debata', 'tylko', 'dłubać', 'zwykła', 'z', 'fanga']
['w', 'moim', 'krnąb

In [ ]:
with_ppmi = check_ppmi(created_ppmi)
print(with_ppmi)
print(np.mean(with_ppmi))

[24.128885803340815, 23.72476015225653, 40.299157522862394, 28.406860920141046, 19.67656976800958, 53.91821330547517, 24.992510619919617, 3.436609248753067, 34.980177196507604, 18.009574572253896, 5.0928098911403925, 5.8254513623165645, 23.6319636759869, 11.417893957885003, 14.310938176706259, 23.160422056579566, 15.76974876778197, 13.638503995043646, 34.82300804258888, 31.075605544871046, 20.565740986181176, 29.56860679165169, 15.9626801625595, 1.4254239365129773, 20.23150332271748, 26.061334857835135, 25.362302327368972, 28.55721017033869, 13.449623120624643, 14.057646268976404, 29.495303558399595, 44.6346417562781, 45.91190106660494, 8.433510339063531, 43.27609798722083, 45.40110091384923, 40.640081989737496, 41.952787975234784, 11.87904384667732, 26.475519580874494, 52.496865303247944, 51.68616938952806, 13.42776385595192, 13.245028479544521, 31.463582863144634, 14.91754806865368, 35.66574816065033, 24.154104376661543, 48.23739110323735, 44.892577241983965, 20.33784745316932, 33.29

In [ ]:
with_bigrams = check_ppmi(created_bigrams)
print(with_bigrams)
print(np.mean(with_bigrams))

[24.128885803340815, 23.72476015225653, 36.042772839144305, 5.087049733879673, 40.299157522862394, 28.406860920141046, 27.83156464968274, 6.503564618300543, 17.829695064475477, 3.436609248753067, 12.948369881372756, 13.638503995043646, 34.82300804258888, 31.075605544871046, 20.790993899861803, 1.4254239365129773, 13.449623120624643, 14.057646268976404, 16.48017445404535, 27.957840290626432, 29.495303558399595, 44.6346417562781, 37.242810130165104, 18.611612939489596, 17.445509009761285, 14.397238230025778, 53.50887769226867, 8.433510339063531, 37.2873819093518, 41.952787975234784, 17.357649391562386, 10.24028362237925, 17.297680711877646, 26.475519580874494, 13.42776385595192, 13.245028479544521, 31.463582863144634, 14.91754806865368, 34.102534371832924, 16.891564846607366, 35.66574816065033, 24.154104376661543, 13.336267895211972, 9.784559757949555, 31.767922764959877, 14.287903046959809, 33.23643582497753, 14.730199120625148, 15.858240490156575, 33.29875343019532, 29.784177606976844,

In [ ]:
with_unigrams = check_ppmi(created_unigrams)
print(with_unigrams)
print(np.mean(with_unigrams))

[3.264073545993771, 4.030415802046238, 0.796835488946669, 5.651280644808056, 0, 4.292769131662385, 10.176645330368723, 16.22186909804742, 20.01134287713226, 2.243317247666569, 37.332603225905295, 2.300219224340811, 0.6451895502090967, 0.6533452913996257, 2.5296386861178504, 2.6991086858506623, 5.149348893739063, 1.6030681224463534, 26.211740313973365, 0, 0, 1.348739938899364, 0, 0, 0, 0.8653669019108775, 4.2271765888562935, 2.2674077879235, 0.5881420714430522, 0, 5.879924466965737, 0.33860768130528923, 15.65936924719725, 11.857535585660425]
5.554267100906353
